In [1]:
import urllib.request
import json
import folium
import requests
from datetime import datetime

def get_coordinates_opencage(area_name, api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json"
    params = {
        'key': api_key,
        'q': area_name,
        'limit': 1
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']
            return (location['lat'], location['lng'])
        else:
            return None
    else:
        return None

def format_time(time_str):
    # Parse the time string to a datetime object
    dt = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    # Format the datetime object to a preferred string format
    return dt.strftime("%Y-%m-%d %H:%M:%S")

# Define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v1/IpawsArchivedAlerts"

# Define a query using parameters 
orderby = "?$orderby=id"                                                     
format = "&$format=json"    

try:
    # Issue API call
    request = urllib.request.urlopen(baseUrl + orderby + format)
    result = request.read()

    # Load the result into a JSON object
    jsonData = json.loads(result.decode())
    
    # Extract the first 10 records
    records = jsonData['IpawsArchivedAlerts'][0:10]
    
    api_key = "091322c2c53048edbe1d12dd1b24c0d2"  # Replace with your actual OpenCage API key

    # Create a map centered on the US
    m = folium.Map(location=[37.0902, -95.7129], zoom_start=5)
    
    for record in records:
        # Extract specific elements (example: identifier, sender, sent date)
        identifier = record['identifier']
        sender = record['sender']
        sent = format_time(record['sent'])  # Format the sent time
       
        # Check if 'info' key exists and is not None
        if record.get('info'):
            event = record['info'][0]['event']
            description = record['info'][0]['description']
           
            # Extract coordinates from searchGeometry if it exists
            searchGeometry = record.get('searchGeometry', None)
            if searchGeometry is not None:
                coordinates = searchGeometry.get('coordinates', None)
            else:
                coordinates = "No coordinates available"
           
            # Check if 'areas' key exists
            if 'areas' in record['info'][0]:
                areas = record['info'][0]['areas'][0]
                area_desc = areas['areaDesc']
                geocode = areas['geocode']
               
                # Check if 'polygon' key exists
                polygon = areas.get('polygon', None)
                if polygon is not None:
                    polygon_coords = polygon.get('coordinates', None)
                else:
                    polygon_coords = "No polygon coordinates available"

                # Get coordinates for the area description using OpenCage
                area_coords = get_coordinates_opencage(area_desc, api_key)
               
                # Fallback to search geometry coordinates if area coordinates are not found
                if not area_coords and coordinates != "No coordinates available":
                    area_coords = coordinates[0][0]

                if area_coords:
                    # Add a marker for the alert
                    folium.Marker(
                        location=area_coords,
                        popup=folium.Popup(f"<b>{event}</b><br>{description}<br>Identifier: {identifier}<br>Sender: {sender}<br>Sent: {sent}", max_width=300),
                        tooltip=identifier,
                        icon=folium.Icon(color='blue', icon='info-sign')
                    ).add_to(m)
                else:
                    print(f"Coordinates not found for area: {area_desc}")
               
                if polygon_coords and polygon_coords != "No polygon coordinates available":
                    # Add a polygon to the map
                    folium.Polygon(
                        locations=polygon_coords[0],
                        color='blue',
                        fill=True,
                        fill_opacity=0.4
                    ).add_to(m)
                else:
                    print(f"No polygon coordinates available for area: {area_desc}")
            else:
                print(f"Record {identifier} does not have 'areas' key.")
        else:
            print(f"Record {identifier} does not have 'info' key.")

    # Add a layer control
    folium.LayerControl().add_to(m)

    # Save the map to an HTML file
    m.save('alerts_map.html')
    print("Map has been created and saved as 'alerts_map.html'")

except urllib.error.HTTPError as e:
    print(f"HTTPError: {e.code} {e.reason}")
except urllib.error.URLError as e:
    print(f"URLError: {e.reason}")
except KeyError as e:
    print(f"KeyError: Missing key {e}")


No polygon coordinates available for area: Coastal Citrus; Coastal Levy
No polygon coordinates available for area: Currituck Sound
No polygon coordinates available for area: Western Prince William Sound
No polygon coordinates available for area: Inland Berkeley
No polygon coordinates available for area: Neosho
Map has been created and saved as 'alerts_map.html'


In [2]:
import urllib.request
import json
import requests
from datetime import datetime

def get_coordinates_opencage(area_name, api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json"
    params = {
        'key': api_key,
        'q': area_name,
        'limit': 1
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']
            return (location['lat'], location['lng'])
        else:
            return None
    else:
        return None

def format_time(time_str):
    # Parse the time string to a datetime object
    dt = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    # Format the datetime object to a preferred string format
    return dt.strftime("%Y-%m-%d %H:%M:%S")

# Define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v1/IpawsArchivedAlerts"

# Define a query using parameters 
orderby = "?$orderby=id"                                                     
format = "&$format=json"    

# Fetch 1000 records
limit = "&$top=1000"

try:
    # Issue API call
    request = urllib.request.urlopen(baseUrl + orderby + format + limit)
    result = request.read()

    # Load the result into a JSON object
    jsonData = json.loads(result.decode())
    
    # Extract the records
    records = jsonData['IpawsArchivedAlerts']
    
    # Counters for records with and without area descriptions and coordinates
    with_area_desc = 0
    without_area_desc = 0
    with_coordinates = 0
    without_coordinates = 0

    for record in records:
        # Check if 'info' key exists and is not None
        if record.get('info'):
            # Check if 'areas' key exists
            if 'areas' in record['info'][0]:
                areas = record['info'][0]['areas'][0]
                area_desc = areas.get('areaDesc', None)
                if area_desc:
                    with_area_desc += 1
                else:
                    without_area_desc += 1

                # Check for coordinates in searchGeometry or area description
                searchGeometry = record.get('searchGeometry', None)
                coordinates = areas.get('polygon', None) or (searchGeometry and searchGeometry.get('coordinates', None))

                if coordinates:
                    with_coordinates += 1
                else:
                    without_coordinates += 1
            else:
                without_area_desc += 1
                without_coordinates += 1
        else:
            without_area_desc += 1
            without_coordinates += 1

    print(f"Total records analyzed: {len(records)}")
    print(f"Records with area description: {with_area_desc}")
    print(f"Records without area description: {without_area_desc}")
    print(f"Records with coordinates: {with_coordinates}")
    print(f"Records without coordinates: {without_coordinates}")

except urllib.error.HTTPError as e:
    print(f"HTTPError: {e.code} {e.reason}")
except urllib.error.URLError as e:
    print(f"URLError: {e.reason}")
except KeyError as e:
    print(f"KeyError: Missing key {e}")


Total records analyzed: 1000
Records with area description: 922
Records without area description: 78
Records with coordinates: 422
Records without coordinates: 578


In [3]:
import urllib.request
import json
import folium
import requests
from datetime import datetime

def get_coordinates_opencage(area_name, api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json"
    params = {
        'key': api_key,
        'q': area_name,
        'limit': 1
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']
            return (location['lat'], location['lng'])
        else:
            return None
    else:
        return None

def format_time(time_str):
    # Parse the time string to a datetime object
    dt = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    # Format the datetime object to a preferred string format
    return dt.strftime("%Y-%m-%d %H:%M:%S")

def is_within_date_range(sent_date, start_date, end_date):
    sent_dt = datetime.strptime(sent_date, "%Y-%m-%dT%H:%M:%S.%fZ")
    return start_date <= sent_dt <= end_date

# Define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v1/IpawsArchivedAlerts"

# Define a query using parameters 
orderby = "?$orderby=id"                                                     
format = "&$format=json"    

# Prompt user for start and end dates
start_date_str = input("Enter the start date (YYYY-MM-DD): ")
end_date_str = input("Enter the end date (YYYY-MM-DD): ")

start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

try:
    # Issue API call
    request = urllib.request.urlopen(baseUrl + orderby + format)
    result = request.read()

    # Load the result into a JSON object
    jsonData = json.loads(result.decode())
    
    # Filter records based on date range
    records = [record for record in jsonData['IpawsArchivedAlerts'] if is_within_date_range(record['sent'], start_date, end_date)]
    
    api_key = "091322c2c53048edbe1d12dd1b24c0d2"  # Replace with your actual OpenCage API key

    # Create a map centered on the US
    m = folium.Map(location=[37.0902, -95.7129], zoom_start=5)
    
    for record in records:
        # Extract specific elements (example: identifier, sender, sent date)
        identifier = record['identifier']
        sender = record['sender']
        sent = format_time(record['sent'])  # Format the sent time
       
        # Check if 'info' key exists and is not None
        if record.get('info'):
            event = record['info'][0]['event']
            description = record['info'][0]['description']
           
            # Extract coordinates from searchGeometry if it exists
            searchGeometry = record.get('searchGeometry', None)
            if searchGeometry is not None:
                coordinates = searchGeometry.get('coordinates', None)
            else:
                coordinates = "No coordinates available"
           
            # Check if 'areas' key exists
            if 'areas' in record['info'][0]:
                areas = record['info'][0]['areas'][0]
                area_desc = areas['areaDesc']
                geocode = areas['geocode']
               
                # Check if 'polygon' key exists
                polygon = areas.get('polygon', None)
                if polygon is not None:
                    polygon_coords = polygon.get('coordinates', None)
                else:
                    polygon_coords = "No polygon coordinates available"

                # Get coordinates for the area description using OpenCage
                area_coords = get_coordinates_opencage(area_desc, api_key)
               
                # Fallback to search geometry coordinates if area coordinates are not found
                if not area_coords and coordinates != "No coordinates available":
                    area_coords = coordinates[0][0]

                if area_coords:
                    # Add a marker for the alert
                    folium.Marker(
                        location=area_coords,
                        popup=folium.Popup(f"<b>{event}</b><br>{description}<br>Identifier: {identifier}<br>Sender: {sender}<br>Sent: {sent}", max_width=300),
                        tooltip=identifier,
                        icon=folium.Icon(color='blue', icon='info-sign')
                    ).add_to(m)
                else:
                    print(f"Coordinates not found for area: {area_desc}")
               
                if polygon_coords and polygon_coords != "No polygon coordinates available":
                    # Add a polygon to the map
                    folium.Polygon(
                        locations=polygon_coords[0],
                        color='blue',
                        fill=True,
                        fill_opacity=0.4
                    ).add_to(m)
                else:
                    print(f"No polygon coordinates available for area: {area_desc}")
            else:
                print(f"Record {identifier} does not have 'areas' key.")
        else:
            print(f"Record {identifier} does not have 'info' key.")

    # Add a layer control
    folium.LayerControl().add_to(m)

    # Save the map to an HTML file
    m.save('alerts_map.html')
    print("Map has been created and saved as 'alerts_map.html'")

except urllib.error.HTTPError as e:
    print(f"HTTPError: {e.code} {e.reason}")
except urllib.error.URLError as e:
    print(f"URLError: {e.reason}")
except KeyError as e:
    print(f"KeyError: Missing key {e}")


Enter the start date (YYYY-MM-DD): 2024-10-01
Enter the end date (YYYY-MM-DD): 2024-12-01
Record urn:oid:2.49.0.1.840.0.950aceba702499550a431cb7a8f7e81c1939f771.015.1.IPAWS does not have 'areas' key.
Map has been created and saved as 'alerts_map.html'


## Markers are not showing

In [5]:
import urllib.request
import json
import folium
import requests
from datetime import datetime

def get_coordinates_opencage(area_name, api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json"
    params = {
        'key': api_key,
        'q': area_name,
        'limit': 1
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']
            return (location['lat'], location['lng'])
        else:
            return None
    else:
        return None

def format_time(time_str):
    # Parse the time string to a datetime object
    dt = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    # Format the datetime object to a preferred string format
    return dt.strftime("%Y-%m-%d %H:%M:%S")

def is_within_date_range(sent_date, start_date, end_date):
    sent_dt = datetime.strptime(sent_date, "%Y-%m-%dT%H:%M:%S.%fZ")
    return start_date <= sent_dt <= end_date

# Define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v1/IpawsArchivedAlerts"

# Define a query using parameters 
orderby = "?$orderby=id"                                                     
format = "&$format=json"    

# Prompt user for start and end dates
start_date_str = input("Enter the start date (YYYY-MM-DD): ")
end_date_str = input("Enter the end date (YYYY-MM-DD): ")

start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

try:
    # Issue API call
    request = urllib.request.urlopen(baseUrl + orderby + format)
    result = request.read()

    # Load the result into a JSON object
    jsonData = json.loads(result.decode())
    
    # Filter records based on date range
    records = [record for record in jsonData['IpawsArchivedAlerts'] if is_within_date_range(record['sent'], start_date, end_date)]
    
    api_key = "091322c2c53048edbe1d12dd1b24c0d2"  # Replace with your actual OpenCage API key

    # Create a map centered on the US
    m = folium.Map(location=[37.0902, -95.7129], zoom_start=5)
    
    for record in records:
        # Extract specific elements (example: identifier, sender, sent date)
        identifier = record['identifier']
        sender = record['sender']
        sent = format_time(record['sent'])  # Format the sent time
       
        # Check if 'info' key exists and is not None
        if record.get('info'):
            event = record['info'][0]['event']
            description = record['info'][0]['description']
           
            # Extract coordinates from searchGeometry if it exists
            searchGeometry = record.get('searchGeometry', None)
            if searchGeometry is not None:
                coordinates = searchGeometry.get('coordinates', None)
            else:
                coordinates = "No coordinates available"
           
            # Check if 'areas' key exists
            if 'areas' in record['info'][0]:
                areas = record['info'][0]['areas'][0]
                area_desc = areas['areaDesc']
                geocode = areas['geocode']
               
                # Check if 'polygon' key exists
                polygon = areas.get('polygon', None)
                if polygon is not None:
                    polygon_coords = polygon.get('coordinates', None)
                else:
                    polygon_coords = "No polygon coordinates available"

                # Get coordinates for the area description using OpenCage
                area_coords = get_coordinates_opencage(area_desc, api_key)
               
                # Fallback to search geometry coordinates if area coordinates are not found
                if not area_coords and coordinates != "No coordinates available":
                    area_coords = coordinates[0][0]

                if area_coords:
                    print(f"Adding marker for area: {area_desc} at {area_coords}")
                    # Add a marker for the alert
                    folium.Marker(
                        location=area_coords,
                        popup=folium.Popup(f"<b>{event}</b><br>{description}<br>Identifier: {identifier}<br>Sender: {sender}<br>Sent: {sent}", max_width=300),
                        tooltip=identifier,
                        icon=folium.Icon(color='blue', icon='info-sign')
                    ).add_to(m)
                else:
                    print(f"Coordinates not found for area: {area_desc}")
               
                if polygon_coords and polygon_coords != "No polygon coordinates available":
                    # Add a polygon to the map
                    folium.Polygon(
                        locations=polygon_coords[0],
                        color='blue',
                        fill=True,
                        fill_opacity=0.4
                    ).add_to(m)
                else:
                    print(f"No polygon coordinates available for area: {area_desc}")
            else:
                print(f"Record {identifier} does not have 'areas' key.")
        else:
            print(f"Record {identifier} does not have 'info' key.")

    # Add a layer control
    folium.LayerControl().add_to(m)

    # Save the map to an HTML file
    m.save('alerts_map.html')
    print("Map has been created and saved as 'alerts_map.html'")

except urllib.error.HTTPError as e:
    print(f"HTTPError: {e.code} {e.reason}")
except urllib.error.URLError as e:
    print(f"URLError: {e.reason}")
except KeyError as e:
    print(f"KeyError: Missing key {e}")


Enter the start date (YYYY-MM-DD): 2024-01-01
Enter the end date (YYYY-MM-DD): 2024-08-01
Map has been created and saved as 'alerts_map.html'


In [6]:
import urllib.request
import json
import folium
import requests
from datetime import datetime

def get_coordinates_opencage(area_name, api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json"
    params = {
        'key': api_key,
        'q': area_name,
        'limit': 1
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']
            return (location['lat'], location['lng'])
        else:
            return None
    else:
        return None

def format_time(time_str):
    # Parse the time string to a datetime object
    dt = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    # Format the datetime object to a preferred string format
    return dt.strftime("%Y-%m-%d %H:%M:%S")

def is_within_date_range(sent_date, start_date, end_date):
    sent_dt = datetime.strptime(sent_date, "%Y-%m-%dT%H:%M:%S.%fZ")
    return start_date <= sent_dt <= end_date

# Define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v1/IpawsArchivedAlerts"

# Define a query using parameters 
orderby = "?$orderby=id"                                                     
format = "&$format=json"    

# Prompt user for start and end dates
start_date_str = input("Enter the start date (YYYY-MM-DD): ")
end_date_str = input("Enter the end date (YYYY-MM-DD): ")

start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

try:
    # Issue API call
    request = urllib.request.urlopen(baseUrl + orderby + format)
    result = request.read()

    # Load the result into a JSON object
    jsonData = json.loads(result.decode())
    
    # Filter records based on date range
    records = [record for record in jsonData['IpawsArchivedAlerts'] if is_within_date_range(record['sent'], start_date, end_date)]
    
    api_key = "091322c2c53048edbe1d12dd1b24c0d2"  # Replace with your actual OpenCage API key

    # Create a map centered on the US
    m = folium.Map(location=[37.0902, -95.7129], zoom_start=5)
    
    for record in records:
        # Extract specific elements (example: identifier, sender, sent date)
        identifier = record['identifier']
        sender = record['sender']
        sent = format_time(record['sent'])  # Format the sent time
       
        # Check if 'info' key exists and is not None
        if record.get('info'):
            event = record['info'][0]['event']
            description = record['info'][0]['description']
           
            # Extract coordinates from searchGeometry if it exists
            searchGeometry = record.get('searchGeometry', None)
            if searchGeometry is not None:
                coordinates = searchGeometry.get('coordinates', None)
            else:
                coordinates = "No coordinates available"
           
            # Check if 'areas' key exists
            if 'areas' in record['info'][0]:
                areas = record['info'][0]['areas'][0]
                area_desc = areas['areaDesc']
                geocode = areas['geocode']
               
                # Check if 'polygon' key exists
                polygon = areas.get('polygon', None)
                if polygon is not None:
                    polygon_coords = polygon.get('coordinates', None)
                else:
                    polygon_coords = "No polygon coordinates available"

                # Get coordinates for the area description using OpenCage
                area_coords = get_coordinates_opencage(area_desc, api_key)
               
                # Fallback to search geometry coordinates if area coordinates are not found
                if not area_coords and coordinates != "No coordinates available":
                    area_coords = coordinates[0][0]

                if area_coords:
                    print(f"Adding marker for area: {area_desc} at {area_coords}")
                    print(f"Event: {event}, Description: {description}, Identifier: {identifier}, Sender: {sender}, Sent: {sent}")
                    # Add a marker for the alert
                    folium.Marker(
                        location=area_coords,
                        popup=folium.Popup(f"<b>{event}</b><br>{description}<br>Identifier: {identifier}<br>Sender: {sender}<br>Sent: {sent}", max_width=300),
                        tooltip=identifier,
                        icon=folium.Icon(color='blue', icon='info-sign')
                    ).add_to(m)
                else:
                    print(f"Coordinates not found for area: {area_desc}")
               
                if polygon_coords and polygon_coords != "No polygon coordinates available":
                    print(f"Adding polygon for area: {area_desc} with coordinates: {polygon_coords[0]}")
                    # Add a polygon to the map
                    folium.Polygon(
                        locations=polygon_coords[0],
                        color='blue',
                        fill=True,
                        fill_opacity=0.4
                    ).add_to(m)
                else:
                    print(f"No polygon coordinates available for area: {area_desc}")
            else:
                print(f"Record {identifier} does not have 'areas' key.")
        else:
            print(f"Record {identifier} does not have 'info' key.")

    # Add a layer control
    folium.LayerControl().add_to(m)

    # Save the map to an HTML file
    m.save('alerts_map.html')
    print("Map has been created and saved as 'alerts_map.html'")

except urllib.error.HTTPError as e:
    print(f"HTTPError: {e.code} {e.reason}")
except urllib.error.URLError as e:
    print(f"URLError: {e.reason}")
except KeyError as e:
    print(f"KeyError: Missing key {e}")


Enter the start date (YYYY-MM-DD): 2023-01-01
Enter the end date (YYYY-MM-DD): 2024-01-01
Record urn:oid:2.49.0.1.840.0.dae5eca17fd90803f2a6157116e6401c6214ba8a.001.1 does not have 'areas' key.
Record urn:oid:2.49.0.1.840.0.9af30e46a673724e5d0adc90d302aae21116bb4c.001.1 does not have 'areas' key.
Record urn:oid:2.49.0.1.840.0.762b9617331cdca9cbc8c2ee81a6d0d53b798403.004.2 does not have 'areas' key.
Record urn:oid:2.49.0.1.840.0.09287f948ad7835684c32097e98cac08a34a6268.002.1 does not have 'areas' key.
Record urn:oid:2.49.0.1.840.0.f9398cae3b30a10815a5e06fbabf7c72f7e2e9e0.001.1 does not have 'areas' key.
Record urn:oid:2.49.0.1.840.0.552f57b1bb9711c06256d932433605a3ad984a97.006.1 does not have 'areas' key.
Record urn:oid:2.49.0.1.840.0.3c1c006d9d990677f0c60a6f04cafaaaa11adcd0.001.4 does not have 'areas' key.
Record urn:oid:2.49.0.1.840.0.f23c2884e139b37fedbb5fcfed1ec18a74642e74.001.1 does not have 'areas' key.
Record urn:oid:2.49.0.1.840.0.ad9e2ea69410232ff0cc5967e52d5494fa1cc077.001.1 d